In [1]:
#데이터전처리 import
import pandas as pd
import numpy as np
from solarpy import irradiance_on_plane
from datetime import datetime
import warnings
from glob import glob
import chardet 
import pickle



#tensorflow 
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.layers
from tensorflow.keras.layers import Conv2D, Dense , Flatten,MaxPooling2D,ZeroPadding2D,MaxPool2D,GlobalMaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras import initializers
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.metrics import Accuracy
import tensorflow.keras.backend as K
from tensorflow.keras.losses import Loss
from tensorflow.keras.activations import selu
#데이터시각
import matplotlib.pyplot as plt
import os
import re
import PIL 
from PIL import Image
import matplotlib.pyplot as plt
import math
import matplotlib




2024-01-29 16:33:22.768231: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
#gpus = tf.config.experimental.list_physical_devices('GPU')
#print("사용 가능한 GPU 장치:", gpus)

In [3]:
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

In [4]:
#gpus = tf.config.list_physical_devices('GPU')
#if gpus:
#  try:
#    # Currently, memory growth needs to be the same across GPUs
#    for gpu in gpus:
#      tf.config.experimental.set_memory_growth(gpu, True)
#    logical_gpus = tf.config.list_logical_devices('GPU')
#    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#  except RuntimeError as e:
#    # Memory growth must be set before GPUs have been initialized
#    print(e)

In [5]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 특정 GPU만 사용하도록 제한
  try:
    tf.config.set_visible_devices(gpus[3], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

print(gpus[3])

PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')


2024-01-29 16:33:24.617544: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-29 16:33:24.619127: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-01-29 16:33:24.906892: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2024-01-29 16:33:24.913741: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2024-01-29 16:33:24.919328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:0d:0

<h1>계산 일사량 계산함수</h1>

In [6]:
def C_Insolation_add(df, lat):
    vnorm = np.array([0, 0, -1])  # 정점을 향하는 평면
    h = 0  # 해수면


    Insolation_list = []
    df['일시'] = pd.to_datetime(df['일시'])

    for i in range(len(df)):
        date = df.iloc[i]['일시']
        Insolation = int(irradiance_on_plane(vnorm, h, date, lat)) * 3600 / 1000000
        Insolation_list.append(Insolation)
    
    df['c일사량'] = Insolation_list
    return df


<h1>데이터로드</h1>
encoding 전부꼬와놔서  chardet로 엔코딩확인하고 불러함

<h2>경로 확인후 엔코딩검사</h2>

In [7]:
path = "/home/youngjun/csv/"

with open(path+'dangjin_obs_data관측.csv', 'rb') as file:
   result = chardet.detect(file.read())
   print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


<h2>dangjin_obs_data관측 열기</h2>

In [8]:
files = sorted(glob(path+'*.csv'))
df = pd.read_csv(path+'dangjin_obs_data관측.csv', encoding='utf-8')

df


,지점,지점명,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위)
0,129,서산,2018-03-01 00:00,3.1,3.6,340.0,96.0,NaN
1,129,서산,2018-03-01 01:00,2.8,0.7,140.0,97.0,NaN
2,129,서산,2018-03-01 02:00,2.6,3.2,320.0,95.0,NaN
3,129,서산,2018-03-01 03:00,2.0,1.9,230.0,97.0,NaN
4,129,서산,2018-03-01 04:00,2.2,2.1,180.0,97.0,NaN
...,...,...,...,...,...,...,...,...
25621,129,서산,2021-01-31 19:00,6.7,1.5,200.0,75.0,8.0
25622,129,서산,2021-01-31 20:00,6.2,0.8,200.0,77.0,8.0
25623,129,서산,2021-01-31 21:00,5.3,0.7,230.0,82.0,8.0
25624,129,서산,2021-01-31 22:00,5.8,1.0,200.0,77.0,8.0


<h2> 데이터결측치확인</h2>

In [9]:
df.shape
print(df.isnull().sum())

지점              0
지점명             0
일시              0
기온(°C)         37
풍속(m/s)        36
풍향(16방위)       36
습도(%)          35
전운량(10분위)    3970
dtype: int64


<h2>데이터열제거<h2>
지점및 지점명은 쓸모가없음

In [10]:
df.drop(df.columns[[0, 1]], axis=1, inplace=True)
original_df=pd.DataFrame(df)
df

,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위)
0,2018-03-01 00:00,3.1,3.6,340.0,96.0,NaN
1,2018-03-01 01:00,2.8,0.7,140.0,97.0,NaN
2,2018-03-01 02:00,2.6,3.2,320.0,95.0,NaN
3,2018-03-01 03:00,2.0,1.9,230.0,97.0,NaN
4,2018-03-01 04:00,2.2,2.1,180.0,97.0,NaN
...,...,...,...,...,...,...
25621,2021-01-31 19:00,6.7,1.5,200.0,75.0,8.0
25622,2021-01-31 20:00,6.2,0.8,200.0,77.0,8.0
25623,2021-01-31 21:00,5.3,0.7,230.0,82.0,8.0
25624,2021-01-31 22:00,5.8,1.0,200.0,77.0,8.0


<h2>위도를 통한계산 일사량 계산</h2>
당진 위도 37.0507

In [11]:
lat = 37.0507 # 서울의 위도
df = C_Insolation_add(df, lat)
df

,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
0,2018-03-01 00:00:00,3.1,3.6,340.0,96.0,NaN,0.0
1,2018-03-01 01:00:00,2.8,0.7,140.0,97.0,NaN,0.0
2,2018-03-01 02:00:00,2.6,3.2,320.0,95.0,NaN,0.0
3,2018-03-01 03:00:00,2.0,1.9,230.0,97.0,NaN,0.0
4,2018-03-01 04:00:00,2.2,2.1,180.0,97.0,NaN,0.0
...,...,...,...,...,...,...,...
25621,2021-01-31 19:00:00,6.7,1.5,200.0,75.0,8.0,0.0
25622,2021-01-31 20:00:00,6.2,0.8,200.0,77.0,8.0,0.0
25623,2021-01-31 21:00:00,5.3,0.7,230.0,82.0,8.0,0.0
25624,2021-01-31 22:00:00,5.8,1.0,200.0,77.0,8.0,0.0


<h2>일사량 데이터 잘들어갔나 확인<h2>

In [12]:
max_value = df['c일사량'].max()
print(max_value)

3.33


<h3> 신뢰도 플래그 만들기</h3>
플래그 데이터 프레임 만들기 데이터 시간 부터 맞나확인함

In [13]:
df['일시'] = pd.to_datetime(df['일시'])
full_range = pd.date_range(start='2018-03-01 00:00:00', end='2021-01-31 23:00:00', freq='H')

# 누락된 시간 찾기
missing_times = full_range.difference(df['일시'])
print(missing_times)

DatetimeIndex(['2018-07-24 11:00:00', '2018-07-24 12:00:00',
               '2018-07-24 13:00:00', '2018-07-24 14:00:00',
               '2018-07-24 15:00:00', '2018-07-24 16:00:00'],
              dtype='datetime64[ns]', freq=None)


<h3>데이터의 시간 결측치 제거</h3>
25625->25632 

In [14]:
full_time_series = pd.date_range(start='2018-03-01 00:00:00', end='2021-01-31 23:00:00', freq='H')
df_full_time_series = pd.DataFrame(full_time_series, columns=['일시'])
df_merged = df_full_time_series.merge(df, on='일시', how='left')
df_merged

,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
0,2018-03-01 00:00:00,3.1,3.6,340.0,96.0,NaN,0.0
1,2018-03-01 01:00:00,2.8,0.7,140.0,97.0,NaN,0.0
2,2018-03-01 02:00:00,2.6,3.2,320.0,95.0,NaN,0.0
3,2018-03-01 03:00:00,2.0,1.9,230.0,97.0,NaN,0.0
4,2018-03-01 04:00:00,2.2,2.1,180.0,97.0,NaN,0.0
...,...,...,...,...,...,...,...
25627,2021-01-31 19:00:00,6.7,1.5,200.0,75.0,8.0,0.0
25628,2021-01-31 20:00:00,6.2,0.8,200.0,77.0,8.0,0.0
25629,2021-01-31 21:00:00,5.3,0.7,230.0,82.0,8.0,0.0
25630,2021-01-31 22:00:00,5.8,1.0,200.0,77.0,8.0,0.0


In [15]:
specific_row = df_merged.loc[df_merged['일시'] == '2018-07-24 15:00:00']
print(specific_row)

                      일시  기온(°C)  풍속(m/s)  풍향(16방위)  습도(%)  전운량(10분위)  c일사량
3495 2018-07-24 15:00:00     NaN      NaN       NaN    NaN        NaN   NaN


<h1> 일시 인덱스로 바꿈</h1>

In [16]:
df_merged.set_index('일시', inplace=True)
df_merged

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2018-03-01 00:00:00,3.1,3.6,340.0,96.0,NaN,0.0
2018-03-01 01:00:00,2.8,0.7,140.0,97.0,NaN,0.0
2018-03-01 02:00:00,2.6,3.2,320.0,95.0,NaN,0.0
2018-03-01 03:00:00,2.0,1.9,230.0,97.0,NaN,0.0
2018-03-01 04:00:00,2.2,2.1,180.0,97.0,NaN,0.0
...,...,...,...,...,...,...
2021-01-31 19:00:00,6.7,1.5,200.0,75.0,8.0,0.0
2021-01-31 20:00:00,6.2,0.8,200.0,77.0,8.0,0.0
2021-01-31 21:00:00,5.3,0.7,230.0,82.0,8.0,0.0


<h3> 데이터 변경점추적</h3>

In [17]:
new_df = df_merged.applymap(lambda x: 1 if pd.notnull(x) else 0)
row_sums = new_df.sum(axis=1)
reliability = row_sums / len(new_df.columns)
new_df['신뢰도'] = reliability
new_df

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량,신뢰도
일시,,,,,,,
2018-03-01 00:00:00,1,1,1,1,0,1,0.833333
2018-03-01 01:00:00,1,1,1,1,0,1,0.833333
2018-03-01 02:00:00,1,1,1,1,0,1,0.833333
2018-03-01 03:00:00,1,1,1,1,0,1,0.833333
2018-03-01 04:00:00,1,1,1,1,0,1,0.833333
...,...,...,...,...,...,...,...
2021-01-31 19:00:00,1,1,1,1,1,1,1.000000
2021-01-31 20:00:00,1,1,1,1,1,1,1.000000
2021-01-31 21:00:00,1,1,1,1,1,1,1.000000


In [18]:
df_merged = df_merged.fillna(0)
df_merged

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2018-03-01 00:00:00,3.1,3.6,340.0,96.0,0.0,0.0
2018-03-01 01:00:00,2.8,0.7,140.0,97.0,0.0,0.0
2018-03-01 02:00:00,2.6,3.2,320.0,95.0,0.0,0.0
2018-03-01 03:00:00,2.0,1.9,230.0,97.0,0.0,0.0
2018-03-01 04:00:00,2.2,2.1,180.0,97.0,0.0,0.0
...,...,...,...,...,...,...
2021-01-31 19:00:00,6.7,1.5,200.0,75.0,8.0,0.0
2021-01-31 20:00:00,6.2,0.8,200.0,77.0,8.0,0.0
2021-01-31 21:00:00,5.3,0.7,230.0,82.0,8.0,0.0


In [19]:
specific_row = df_merged.loc['2018-07-24 15:00:00']
print(specific_row)

기온(°C)       0.0
풍속(m/s)      0.0
풍향(16방위)     0.0
습도(%)        0.0
전운량(10분위)    0.0
c일사량         0.0
Name: 2018-07-24 15:00:00, dtype: float64


0이상의 일사량만 가져옴

In [20]:
filtered_df = df_merged[df_merged['c일사량'] > 0]
filtered_df

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2018-03-01 07:00:00,2.2,6.4,290.0,86.0,0.0,0.0504
2018-03-01 08:00:00,1.0,7.7,320.0,82.0,0.0,0.5652
2018-03-01 09:00:00,0.3,8.9,320.0,71.0,0.0,1.2204
2018-03-01 10:00:00,0.6,7.9,320.0,63.0,0.0,1.7748
2018-03-01 11:00:00,0.5,9.1,320.0,58.0,0.0,2.1348
...,...,...,...,...,...,...
2021-01-31 12:00:00,9.5,3.6,180.0,50.0,6.0,1.6884
2021-01-31 13:00:00,9.1,2.9,200.0,49.0,7.0,1.5696
2021-01-31 14:00:00,11.0,5.1,200.0,58.0,8.0,1.2348


new_df=마스킹 filtered_df=인풋

In [21]:
valid_dates = filtered_df[filtered_df['c일사량'] >= 0].index
new_df = new_df.loc[valid_dates]
new_df

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량,신뢰도
일시,,,,,,,
2018-03-01 07:00:00,1,1,1,1,0,1,0.833333
2018-03-01 08:00:00,1,1,1,1,0,1,0.833333
2018-03-01 09:00:00,1,1,1,1,0,1,0.833333
2018-03-01 10:00:00,1,1,1,1,0,1,0.833333
2018-03-01 11:00:00,1,1,1,1,0,1,0.833333
...,...,...,...,...,...,...,...
2021-01-31 12:00:00,1,1,1,1,1,1,1.000000
2021-01-31 13:00:00,1,1,1,1,1,1,1.000000
2021-01-31 14:00:00,1,1,1,1,1,1,1.000000


<h1>데이터 전처리</h1>

<h1>데이터 스케일링 및 주기성제공</h1>

<h1>정답데이터</h1>
'/home/youngjun/csv/ydata/2021_당진_일사량.csv',
 '/home/youngjun/csv/ydata/2018_당진_일사량.csv',
 '/home/youngjun/csv/ydata/2019_당진_일사량.csv',
 '/home/youngjun/csv/ydata/2020_당진_일사량.csv'
 총 4개의 파일이라 합치고 써야함

In [22]:
path = '/home/youngjun/csv/ydata/*.csv'
csv_files = glob(path)
df_list = []

for file in csv_files:
    
     print(file)
     df = pd.read_csv(file, encoding='EUC-KR')  # 혹은 해당하는 인코딩을 사용합니다.
     df_list.append(df)
    
combined_df = pd.concat(df_list, ignore_index=True)    
combined_df['일시'] = pd.to_datetime(combined_df['일시'])
combined_df = combined_df.sort_values('일시', ascending=True)
combined_df

/home/youngjun/csv/ydata/2021_당진_일사량.csv
/home/youngjun/csv/ydata/2018_당진_일사량.csv
/home/youngjun/csv/ydata/2019_당진_일사량.csv
/home/youngjun/csv/ydata/2020_당진_일사량.csv


,지점,지점명,일시,일사(MJ/m2)
341,129,서산,2018-01-01 08:00:00,0.00
342,129,서산,2018-01-01 09:00:00,0.18
343,129,서산,2018-01-01 10:00:00,0.45
344,129,서산,2018-01-01 11:00:00,0.90
345,129,서산,2018-01-01 12:00:00,1.20
...,...,...,...,...
336,129,서산,2021-01-31 14:00:00,1.74
337,129,서산,2021-01-31 15:00:00,1.37
338,129,서산,2021-01-31 16:00:00,0.73
339,129,서산,2021-01-31 17:00:00,0.24


In [23]:
combined_df = combined_df.drop(['지점','지점명'], axis=1)
combined_df.set_index('일시', inplace=True)
combined_df

,일사(MJ/m2)
일시,
2018-01-01 08:00:00,0.00
2018-01-01 09:00:00,0.18
2018-01-01 10:00:00,0.45
2018-01-01 11:00:00,0.90
2018-01-01 12:00:00,1.20
...,...
2021-01-31 14:00:00,1.74
2021-01-31 15:00:00,1.37
2021-01-31 16:00:00,0.73


In [24]:
combined_df = combined_df.loc['2018-03-01 00:00:00':'2021-01-31 23:00']
combined_df

,일사(MJ/m2)
일시,
2018-03-01 08:00:00,0.05
2018-03-01 09:00:00,0.69
2018-03-01 10:00:00,1.43
2018-03-01 11:00:00,2.11
2018-03-01 12:00:00,2.57
...,...
2021-01-31 14:00:00,1.74
2021-01-31 15:00:00,1.37
2021-01-31 16:00:00,0.73


In [25]:
filtered_df
new_filtered_df=filtered_df.copy()
new_filtered_df

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2018-03-01 07:00:00,2.2,6.4,290.0,86.0,0.0,0.0504
2018-03-01 08:00:00,1.0,7.7,320.0,82.0,0.0,0.5652
2018-03-01 09:00:00,0.3,8.9,320.0,71.0,0.0,1.2204
2018-03-01 10:00:00,0.6,7.9,320.0,63.0,0.0,1.7748
2018-03-01 11:00:00,0.5,9.1,320.0,58.0,0.0,2.1348
...,...,...,...,...,...,...
2021-01-31 12:00:00,9.5,3.6,180.0,50.0,6.0,1.6884
2021-01-31 13:00:00,9.1,2.9,200.0,49.0,7.0,1.5696
2021-01-31 14:00:00,11.0,5.1,200.0,58.0,8.0,1.2348


In [26]:
common_indices = combined_df.index.intersection(new_filtered_df.index)
filtered_a = combined_df.loc[common_indices]
filtered_a

,일사(MJ/m2)
일시,
2018-03-01 08:00:00,0.05
2018-03-01 09:00:00,0.69
2018-03-01 10:00:00,1.43
2018-03-01 11:00:00,2.11
2018-03-01 12:00:00,2.57
...,...
2021-01-31 12:00:00,1.75
2021-01-31 13:00:00,1.63
2021-01-31 14:00:00,1.74


In [27]:
filtered_b = new_filtered_df.loc[common_indices]
filtered_b

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2018-03-01 08:00:00,1.0,7.7,320.0,82.0,0.0,0.5652
2018-03-01 09:00:00,0.3,8.9,320.0,71.0,0.0,1.2204
2018-03-01 10:00:00,0.6,7.9,320.0,63.0,0.0,1.7748
2018-03-01 11:00:00,0.5,9.1,320.0,58.0,0.0,2.1348
2018-03-01 12:00:00,0.7,6.7,320.0,60.0,0.0,2.2572
...,...,...,...,...,...,...
2021-01-31 12:00:00,9.5,3.6,180.0,50.0,6.0,1.6884
2021-01-31 13:00:00,9.1,2.9,200.0,49.0,7.0,1.5696
2021-01-31 14:00:00,11.0,5.1,200.0,58.0,8.0,1.2348


In [28]:
def normalize_humidity(humidity):
    return (humidity - min_humidity) / (max_humidity - min_humidity) * 2 - 1

In [29]:
filtered_b = filtered_b.reset_index()
filtered_b['일시'] = pd.to_datetime(filtered_b['일시'])
filtered_b['hour'] = filtered_b['일시'].dt.hour
filtered_b['hour_cos'] = np.cos(2 * np.pi * filtered_b['hour'] / 24)
filtered_b['hour_sin'] = np.sin(2 * np.pi * filtered_b['hour'] / 24)

filtered_b['month'] = filtered_b['일시'].dt.month
filtered_b['month_cos'] = np.cos(2 * np.pi * filtered_b['month'] / 12)
filtered_b['month_sin'] = np.sin(2 * np.pi * filtered_b['month'] / 12)

filtered_b['year'] = filtered_b['일시'].dt.year
filtered_b['year']=filtered_b['year']-2018

In [30]:
min_humidity = filtered_b['습도(%)'].min()
max_humidity = filtered_b['습도(%)'].max()
filtered_b['정규화된습도'] = filtered_b['습도(%)'].apply(normalize_humidity)

min_humidity = filtered_b['전운량(10분위)'].min()
max_humidity = filtered_b['전운량(10분위)'].max()
filtered_b['정규화된전운량'] = filtered_b['전운량(10분위)'].apply(normalize_humidity)
filtered_b

,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량,hour,hour_cos,hour_sin,month,month_cos,month_sin,year,정규화된습도,정규화된전운량
0,2018-03-01 08:00:00,1.0,7.7,320.0,82.0,0.0,0.5652,8,-0.500000,8.660254e-01,3,6.123234e-17,1.0,0,0.656566,-1.0
1,2018-03-01 09:00:00,0.3,8.9,320.0,71.0,0.0,1.2204,9,-0.707107,7.071068e-01,3,6.123234e-17,1.0,0,0.434343,-1.0
2,2018-03-01 10:00:00,0.6,7.9,320.0,63.0,0.0,1.7748,10,-0.866025,5.000000e-01,3,6.123234e-17,1.0,0,0.272727,-1.0
3,2018-03-01 11:00:00,0.5,9.1,320.0,58.0,0.0,2.1348,11,-0.965926,2.588190e-01,3,6.123234e-17,1.0,0,0.171717,-1.0
4,2018-03-01 12:00:00,0.7,6.7,320.0,60.0,0.0,2.2572,12,-1.000000,1.224647e-16,3,6.123234e-17,1.0,0,0.212121,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11883,2021-01-31 12:00:00,9.5,3.6,180.0,50.0,6.0,1.6884,12,-1.000000,1.224647e-16,1,8.660254e-01,0.5,3,0.010101,0.2
11884,2021-01-31 13:00:00,9.1,2.9,200.0,49.0,7.0,1.5696,13,-0.965926,-2.588190e-01,1,8.660254e-01,0.5,3,-0.010101,0.4
11885,2021-01-31 14:00:00,11.0,5.1,200.0,58.0,8.0,1.2348,14,-0.866025,-5.000000e-01,1,8.660254e-01,0.5,3,0.171717,0.6
11886,2021-01-31 15:00:00,10.4,5.0,230.0,60.0,8.0,0.7344,15,-0.707107,-7.071068e-01,1,8.660254e-01,0.5,3,0.212121,0.6


In [31]:
wv = filtered_b.pop('풍속(m/s)')


# 'wd (deg)' 열을 라디안으로 변환
wd_rad = filtered_b['풍향(16방위)'] * np.pi / 180

# 바람 벡터의 x 및 y 구성 요소 계산 및 추가
filtered_b['Wx'] = wv * np.cos(wd_rad)
filtered_b['Wy'] = wv * np.sin(wd_rad)


filtered_b

,일시,기온(°C),풍향(16방위),습도(%),전운량(10분위),c일사량,hour,hour_cos,hour_sin,month,month_cos,month_sin,year,정규화된습도,정규화된전운량,Wx,Wy
0,2018-03-01 08:00:00,1.0,320.0,82.0,0.0,0.5652,8,-0.500000,8.660254e-01,3,6.123234e-17,1.0,0,0.656566,-1.0,5.898542,-4.949465e+00
1,2018-03-01 09:00:00,0.3,320.0,71.0,0.0,1.2204,9,-0.707107,7.071068e-01,3,6.123234e-17,1.0,0,0.434343,-1.0,6.817796,-5.720810e+00
2,2018-03-01 10:00:00,0.6,320.0,63.0,0.0,1.7748,10,-0.866025,5.000000e-01,3,6.123234e-17,1.0,0,0.272727,-1.0,6.051751,-5.078022e+00
3,2018-03-01 11:00:00,0.5,320.0,58.0,0.0,2.1348,11,-0.965926,2.588190e-01,3,6.123234e-17,1.0,0,0.171717,-1.0,6.971004,-5.849367e+00
4,2018-03-01 12:00:00,0.7,320.0,60.0,0.0,2.2572,12,-1.000000,1.224647e-16,3,6.123234e-17,1.0,0,0.212121,-1.0,5.132498,-4.306677e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11883,2021-01-31 12:00:00,9.5,180.0,50.0,6.0,1.6884,12,-1.000000,1.224647e-16,1,8.660254e-01,0.5,3,0.010101,0.2,-3.600000,4.408728e-16
11884,2021-01-31 13:00:00,9.1,200.0,49.0,7.0,1.5696,13,-0.965926,-2.588190e-01,1,8.660254e-01,0.5,3,-0.010101,0.4,-2.725109,-9.918584e-01
11885,2021-01-31 14:00:00,11.0,200.0,58.0,8.0,1.2348,14,-0.866025,-5.000000e-01,1,8.660254e-01,0.5,3,0.171717,0.6,-4.792432,-1.744303e+00
11886,2021-01-31 15:00:00,10.4,230.0,60.0,8.0,0.7344,15,-0.707107,-7.071068e-01,1,8.660254e-01,0.5,3,0.212121,0.6,-3.213938,-3.830222e+00


In [32]:
filtered_b.set_index('일시', inplace=True)

In [33]:
filtered_b.drop(columns=['hour','month','습도(%)','전운량(10분위)','풍향(16방위)'], inplace=True)


filtered_b

,기온(°C),c일사량,hour_cos,hour_sin,month_cos,month_sin,year,정규화된습도,정규화된전운량,Wx,Wy
일시,,,,,,,,,,,
2018-03-01 08:00:00,1.0,0.5652,-0.500000,8.660254e-01,6.123234e-17,1.0,0,0.656566,-1.0,5.898542,-4.949465e+00
2018-03-01 09:00:00,0.3,1.2204,-0.707107,7.071068e-01,6.123234e-17,1.0,0,0.434343,-1.0,6.817796,-5.720810e+00
2018-03-01 10:00:00,0.6,1.7748,-0.866025,5.000000e-01,6.123234e-17,1.0,0,0.272727,-1.0,6.051751,-5.078022e+00
2018-03-01 11:00:00,0.5,2.1348,-0.965926,2.588190e-01,6.123234e-17,1.0,0,0.171717,-1.0,6.971004,-5.849367e+00
2018-03-01 12:00:00,0.7,2.2572,-1.000000,1.224647e-16,6.123234e-17,1.0,0,0.212121,-1.0,5.132498,-4.306677e+00
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-31 12:00:00,9.5,1.6884,-1.000000,1.224647e-16,8.660254e-01,0.5,3,0.010101,0.2,-3.600000,4.408728e-16
2021-01-31 13:00:00,9.1,1.5696,-0.965926,-2.588190e-01,8.660254e-01,0.5,3,-0.010101,0.4,-2.725109,-9.918584e-01
2021-01-31 14:00:00,11.0,1.2348,-0.866025,-5.000000e-01,8.660254e-01,0.5,3,0.171717,0.6,-4.792432,-1.744303e+00


In [34]:
# 데이터프레임 df의 길이(행의 수)를 계산
length = len(filtered_b)

# 분할 지점 계산 (80% 지점)
split_point = int(length * 0.8)

# 데이터프레임을 학습 세트와 검증 세트로 분할
x_train_df = filtered_b[:split_point]
x_valid_df = filtered_b[split_point:]


In [35]:
x_train_df

,기온(°C),c일사량,hour_cos,hour_sin,month_cos,month_sin,year,정규화된습도,정규화된전운량,Wx,Wy
일시,,,,,,,,,,,
2018-03-01 08:00:00,1.0,0.5652,-0.500000,8.660254e-01,6.123234e-17,1.000000e+00,0,0.656566,-1.0,5.898542,-4.949465
2018-03-01 09:00:00,0.3,1.2204,-0.707107,7.071068e-01,6.123234e-17,1.000000e+00,0,0.434343,-1.0,6.817796,-5.720810
2018-03-01 10:00:00,0.6,1.7748,-0.866025,5.000000e-01,6.123234e-17,1.000000e+00,0,0.272727,-1.0,6.051751,-5.078022
2018-03-01 11:00:00,0.5,2.1348,-0.965926,2.588190e-01,6.123234e-17,1.000000e+00,0,0.171717,-1.0,6.971004,-5.849367
2018-03-01 12:00:00,0.7,2.2572,-1.000000,1.224647e-16,6.123234e-17,1.000000e+00,0,0.212121,-1.0,5.132498,-4.306677
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-24 08:00:00,24.4,1.6992,-0.500000,8.660254e-01,-1.000000e+00,1.224647e-16,2,0.131313,1.0,-3.570832,1.299677
2020-06-24 09:00:00,23.6,2.3616,-0.707107,7.071068e-01,-1.000000e+00,1.224647e-16,2,0.252525,1.0,-3.523804,2.956823
2020-06-24 10:00:00,21.1,2.8800,-0.866025,5.000000e-01,-1.000000e+00,1.224647e-16,2,0.676768,1.0,-0.923454,2.537170


In [36]:
x_valid_df

,기온(°C),c일사량,hour_cos,hour_sin,month_cos,month_sin,year,정규화된습도,정규화된전운량,Wx,Wy
일시,,,,,,,,,,,
2020-06-24 13:00:00,23.8,3.2112,-0.965926,-2.588190e-01,-1.000000,1.224647e-16,2,0.373737,1.0,-3.758770,1.368081e+00
2020-06-24 14:00:00,23.6,2.8800,-0.866025,-5.000000e-01,-1.000000,1.224647e-16,2,0.454545,1.0,-3.382893,1.231273e+00
2020-06-24 15:00:00,22.2,2.3616,-0.707107,-7.071068e-01,-1.000000,1.224647e-16,2,0.717172,0.8,-3.288924,1.197071e+00
2020-06-24 16:00:00,22.2,1.6992,-0.500000,-8.660254e-01,-1.000000,1.224647e-16,2,0.656566,1.0,-3.194955,1.162868e+00
2020-06-24 17:00:00,22.8,0.9684,-0.258819,-9.659258e-01,-1.000000,1.224647e-16,2,0.616162,0.8,-3.500000,4.286264e-16
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-31 12:00:00,9.5,1.6884,-1.000000,1.224647e-16,0.866025,5.000000e-01,3,0.010101,0.2,-3.600000,4.408728e-16
2021-01-31 13:00:00,9.1,1.5696,-0.965926,-2.588190e-01,0.866025,5.000000e-01,3,-0.010101,0.4,-2.725109,-9.918584e-01
2021-01-31 14:00:00,11.0,1.2348,-0.866025,-5.000000e-01,0.866025,5.000000e-01,3,0.171717,0.6,-4.792432,-1.744303e+00


In [37]:
new_df
filtered_c = new_df.loc[common_indices]
filtered_c

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량,신뢰도
일시,,,,,,,
2018-03-01 08:00:00,1,1,1,1,0,1,0.833333
2018-03-01 09:00:00,1,1,1,1,0,1,0.833333
2018-03-01 10:00:00,1,1,1,1,0,1,0.833333
2018-03-01 11:00:00,1,1,1,1,0,1,0.833333
2018-03-01 12:00:00,1,1,1,1,0,1,0.833333
...,...,...,...,...,...,...,...
2021-01-31 12:00:00,1,1,1,1,1,1,1.000000
2021-01-31 13:00:00,1,1,1,1,1,1,1.000000
2021-01-31 14:00:00,1,1,1,1,1,1,1.000000


In [38]:
#filtered_c.drop(columns=['기온(°C)','풍속(m/s)','풍향(16방위)','습도(%)','전운량(10분위)','c일사량'], inplace=True)
filtered_c.drop(columns=['신뢰도'], inplace=True)
filtered_c

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2018-03-01 08:00:00,1,1,1,1,0,1
2018-03-01 09:00:00,1,1,1,1,0,1
2018-03-01 10:00:00,1,1,1,1,0,1
2018-03-01 11:00:00,1,1,1,1,0,1
2018-03-01 12:00:00,1,1,1,1,0,1
...,...,...,...,...,...,...
2021-01-31 12:00:00,1,1,1,1,1,1
2021-01-31 13:00:00,1,1,1,1,1,1
2021-01-31 14:00:00,1,1,1,1,1,1


In [39]:
m_train_df = filtered_c[:split_point]
m_valid_df = filtered_c[split_point:]
m_train_df

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2018-03-01 08:00:00,1,1,1,1,0,1
2018-03-01 09:00:00,1,1,1,1,0,1
2018-03-01 10:00:00,1,1,1,1,0,1
2018-03-01 11:00:00,1,1,1,1,0,1
2018-03-01 12:00:00,1,1,1,1,0,1
...,...,...,...,...,...,...
2020-06-24 08:00:00,1,1,1,1,1,1
2020-06-24 09:00:00,1,1,1,1,1,1
2020-06-24 10:00:00,1,1,1,1,1,1


In [40]:
m_valid_df

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2020-06-24 13:00:00,1,1,1,1,1,1
2020-06-24 14:00:00,1,1,1,1,1,1
2020-06-24 15:00:00,1,1,1,1,1,1
2020-06-24 16:00:00,1,1,1,1,1,1
2020-06-24 17:00:00,1,1,1,1,1,1
...,...,...,...,...,...,...
2021-01-31 12:00:00,1,1,1,1,1,1
2021-01-31 13:00:00,1,1,1,1,1,1
2021-01-31 14:00:00,1,1,1,1,1,1


In [41]:
filtered_a

,일사(MJ/m2)
일시,
2018-03-01 08:00:00,0.05
2018-03-01 09:00:00,0.69
2018-03-01 10:00:00,1.43
2018-03-01 11:00:00,2.11
2018-03-01 12:00:00,2.57
...,...
2021-01-31 12:00:00,1.75
2021-01-31 13:00:00,1.63
2021-01-31 14:00:00,1.74


In [42]:
y_train_df = filtered_a[:split_point]
y_valid_df = filtered_a[split_point:]
y_train_df

,일사(MJ/m2)
일시,
2018-03-01 08:00:00,0.05
2018-03-01 09:00:00,0.69
2018-03-01 10:00:00,1.43
2018-03-01 11:00:00,2.11
2018-03-01 12:00:00,2.57
...,...
2020-06-24 08:00:00,0.11
2020-06-24 09:00:00,0.30
2020-06-24 10:00:00,0.30


In [43]:
y_valid_df

,일사(MJ/m2)
일시,
2020-06-24 13:00:00,0.54
2020-06-24 14:00:00,0.36
2020-06-24 15:00:00,0.26
2020-06-24 16:00:00,0.20
2020-06-24 17:00:00,0.25
...,...
2021-01-31 12:00:00,1.75
2021-01-31 13:00:00,1.63
2021-01-31 14:00:00,1.74


In [44]:
x_train_df

,기온(°C),c일사량,hour_cos,hour_sin,month_cos,month_sin,year,정규화된습도,정규화된전운량,Wx,Wy
일시,,,,,,,,,,,
2018-03-01 08:00:00,1.0,0.5652,-0.500000,8.660254e-01,6.123234e-17,1.000000e+00,0,0.656566,-1.0,5.898542,-4.949465
2018-03-01 09:00:00,0.3,1.2204,-0.707107,7.071068e-01,6.123234e-17,1.000000e+00,0,0.434343,-1.0,6.817796,-5.720810
2018-03-01 10:00:00,0.6,1.7748,-0.866025,5.000000e-01,6.123234e-17,1.000000e+00,0,0.272727,-1.0,6.051751,-5.078022
2018-03-01 11:00:00,0.5,2.1348,-0.965926,2.588190e-01,6.123234e-17,1.000000e+00,0,0.171717,-1.0,6.971004,-5.849367
2018-03-01 12:00:00,0.7,2.2572,-1.000000,1.224647e-16,6.123234e-17,1.000000e+00,0,0.212121,-1.0,5.132498,-4.306677
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-24 08:00:00,24.4,1.6992,-0.500000,8.660254e-01,-1.000000e+00,1.224647e-16,2,0.131313,1.0,-3.570832,1.299677
2020-06-24 09:00:00,23.6,2.3616,-0.707107,7.071068e-01,-1.000000e+00,1.224647e-16,2,0.252525,1.0,-3.523804,2.956823
2020-06-24 10:00:00,21.1,2.8800,-0.866025,5.000000e-01,-1.000000e+00,1.224647e-16,2,0.676768,1.0,-0.923454,2.537170


In [45]:
m_train_df

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2018-03-01 08:00:00,1,1,1,1,0,1
2018-03-01 09:00:00,1,1,1,1,0,1
2018-03-01 10:00:00,1,1,1,1,0,1
2018-03-01 11:00:00,1,1,1,1,0,1
2018-03-01 12:00:00,1,1,1,1,0,1
...,...,...,...,...,...,...
2020-06-24 08:00:00,1,1,1,1,1,1
2020-06-24 09:00:00,1,1,1,1,1,1
2020-06-24 10:00:00,1,1,1,1,1,1


In [46]:
new_order = ['기온(°C)', 'c일사량', 'hour_cos', 'hour_sin', 'month_cos', 'month_sin', 'year', 'humy', 'cloud', 'Wx', 'Wy']
m_train_dfcp=m_train_df.copy()
m_train_dfcp = m_train_dfcp.reindex(columns=new_order)
m_train_dfcp['humy']=m_train_df['습도(%)'].apply(lambda x: 1 if x == 1 else 0)
m_train_dfcp['cloud']=m_train_df['전운량(10분위)'].apply(lambda x: 1 if x == 1 else 0)

mask = (m_train_df['풍속(m/s)'] == 0) | (m_train_df['풍향(16방위)'] == 0)
m_train_dfcp['Wx'] = (~mask).astype(int)
m_train_dfcp['Wy'] = (~mask).astype(int)

m_train_dfcp['hour_cos'] = 1
m_train_dfcp['hour_sin'] = 1
m_train_dfcp['month_cos'] = 1
m_train_dfcp['month_sin'] = 1
m_train_dfcp['year'] = 1
m_train_dfcp



,기온(°C),c일사량,hour_cos,hour_sin,month_cos,month_sin,year,humy,cloud,Wx,Wy
일시,,,,,,,,,,,
2018-03-01 08:00:00,1,1,1,1,1,1,1,1,0,1,1
2018-03-01 09:00:00,1,1,1,1,1,1,1,1,0,1,1
2018-03-01 10:00:00,1,1,1,1,1,1,1,1,0,1,1
2018-03-01 11:00:00,1,1,1,1,1,1,1,1,0,1,1
2018-03-01 12:00:00,1,1,1,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-24 08:00:00,1,1,1,1,1,1,1,1,1,1,1
2020-06-24 09:00:00,1,1,1,1,1,1,1,1,1,1,1
2020-06-24 10:00:00,1,1,1,1,1,1,1,1,1,1,1


In [47]:
filtered_c

,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),c일사량
일시,,,,,,
2018-03-01 08:00:00,1,1,1,1,0,1
2018-03-01 09:00:00,1,1,1,1,0,1
2018-03-01 10:00:00,1,1,1,1,0,1
2018-03-01 11:00:00,1,1,1,1,0,1
2018-03-01 12:00:00,1,1,1,1,0,1
...,...,...,...,...,...,...
2021-01-31 12:00:00,1,1,1,1,1,1
2021-01-31 13:00:00,1,1,1,1,1,1
2021-01-31 14:00:00,1,1,1,1,1,1


In [48]:
#weights_array=np.array(filtered_c)
#weights_array=weights_array.flatten()
#weights_array

In [49]:

import shap

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [50]:
class SharedState:
    def __init__(self):
        self.w_value = None
        
shared_state = SharedState()

class W_mse(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
        
        
    def call(self,y_true,y_pred):
        weights = shared_state.w_value
        error = K.square(y_pred - y_true)
        
        
        weighted_error = error * weights
        
        sum_weighted_error = K.sum(weighted_error)
        # 가중치의 합으로 나누어 가중 오차율을 계산합니다.
        wmse = sum_weighted_error / K.sum(weights)
        return wmse
    
class WeightedLossCallback3(tf.keras.callbacks.Callback):      
    def __init__(self, batch_size, weights_array,x_train,threshold):
        super().__init__()
        self.batch_size = batch_size
        self.weights_array_buf = weights_array
        self.weights_array=None
        self.batch_index = 0
        self.last_epoch_loss = None
        self.x_train=x_train
        self.threshold=threshold
        
    def get_batch_index(self):
        start=self.batch_index*self.batch_size
        end=start+self.batch_size
        
        current_weights = self.weights_array[start:end]
        
        return current_weights
    
    def on_epoch_begin(self, epoch, logs=None):
        # 에폭이 시작될 때 배치 인덱스를 0으로 초기화
        
        self.batch_index = 0
        if self.last_epoch_loss is not None:
            if self.last_epoch_loss < self.threshold:
                # Loss가 임계값보다 낮을 때의 로직 (예: SHAP 값 로직)
                # self.weights_array를 업데이트
                print("Shape wmse")
                model = self.model
                
                explainer = shap.DeepExplainer(model, self.x_train)
                shap_values = explainer.shap_values(self.x_train)
                shap_values = np.array(shap_values)
                shap_values = np.squeeze(shap_values, axis=0)
                
                abs_shap_values = np.abs(shap_values)
                average_abs_shap_values = np.mean(abs_shap_values, axis=0)

                normalized_shap_values = average_abs_shap_values / np.sum(average_abs_shap_values)
                contributions_per_sample = self.weights_array_buf*normalized_shap_values
                final_confidences = np.sum(contributions_per_sample, axis=1)
                print(final_confidences)
                self.weights_array=tf.cast(final_confidences, tf.float32)
                
            else:
                print("nomal wmse")

                # Loss가 임계값보다 높을 때의 로직 (예: 마스크 맵만 사용)
                # self.weights_array를 업데이트
                active_features_per_sample = np.sum(self.weights_array_buf, axis=1)
                normalized_confidences = active_features_per_sample / 11.0
                self.weights_array=tf.cast(normalized_confidences, tf.float32)
        else:
                active_features_per_sample = np.sum(self.weights_array_buf, axis=1)
                normalized_confidences = active_features_per_sample / 11.0
                self.weights_array=tf.cast(normalized_confidences, tf.float32)
        
    def on_batch_begin(self,batch,logs=None):
        shared_state.w_value=self.get_batch_index()
       # print(shared_state.w_value)
        self.batch_index += 1

    def on_epoch_end(self, epoch, logs=None):
        # 에폭 종료 시 loss 값을 저장
        self.last_epoch_loss = logs.get('loss')

In [51]:
m_train_dfcp_num=m_train_dfcp.to_numpy()
x_train_df_num=x_train_df.to_numpy()

In [52]:
#weighted_loss_callback = WeightedLossCallback(32, m_train_df)
WeightedLossCallback_loss=WeightedLossCallback3(32,m_train_dfcp_num,x_train_df_num,6.8)

In [53]:
model=tf.keras.Sequential()
model.add(Dense(64, input_shape=(11,), activation='tanh',kernel_regularizer=l2(0.01)))
model.add(Dense(128, activation='tanh',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3)) # 50% 드롭아웃 적용
model.add(Dense(256, activation='tanh',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3)) # 50% 드롭아웃 적용
model.add(Dense(512, activation='tanh',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5) )# 50% 드롭아웃 적
#model.add(Dense(1024, activation='tanh',kernel_regularizer=l2(0.01)))
#model.add(Dropout(0.5) )
#model.add(Dense(512, activation='tanh',kernel_regularizer=l2(0.01)))
#model.add(Dropout(0.5) )
model.add(Dense(256, activation='tanh',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5)) # 50% 드롭아웃 적용
model.add(Dense(128, activation='tanh',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3)) # 50% 드롭아웃 적용
model.add(Dense(64, activation='tanh',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))  # 50% 드롭아웃 적용
#model.add(Dense(32, activation='tanh'))
#model.add(Dense(16, activation='tanh'))
#model.add(Dense(8, activation='tanh'))
model.add(Dense(1))


#model.compile(optimizer='adam', loss=lambda y_true, y_pred: weighted_mse(y_true, y_pred, get_batch_weights(weighted_loss_callback.batch_index, 64, weights_array)),metrics=['mse'])
#model.compile(optimizer='adam', loss=weighted_loss(0, 32, weights_array), metrics=['mse'])

#model.compile(loss='huber', optimizer='adam', metrics=['mse'])
model.compile(loss=W_mse(), optimizer='adam', metrics=['mse'])
model.summary()


hist = model.fit(x_train_df, y_train_df, epochs=80, batch_size=32,callbacks=[WeightedLossCallback_loss],validation_split=0.2)
#hist = model.fit(x_train_df, y_train_df, epochs=100, batch_size=32,validation_split=0.2)

2024-01-29 16:33:39.237441: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 16:33:39.240993: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-29 16:33:39.244869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2024-01-29 16:33:39.244927: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-01-29 16:33:39.244964: I tensorflow/stream_executor/platform

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                768       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0

2024-01-29 16:33:41.170049: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


238/238 [==============================] - 3s 9ms/step - loss: 59.9630 - mse: 1.4925 - val_loss: 18.5089 - val_mse: 0.1983
Epoch 2/80
nomal wmse
238/238 [==============================] - 2s 7ms/step - loss: 30.4424 - mse: 0.5708 - val_loss: 20.4524 - val_mse: 0.2683
Epoch 3/80
nomal wmse
238/238 [==============================] - 2s 7ms/step - loss: 24.7736 - mse: 0.4026 - val_loss: 20.1143 - val_mse: 0.2690
Epoch 4/80
nomal wmse
238/238 [==============================] - 2s 7ms/step - loss: 21.8238 - mse: 0.3233 - val_loss: 16.0870 - val_mse: 0.1571
Epoch 5/80
nomal wmse
238/238 [==============================] - 2s 7ms/step - loss: 20.6001 - mse: 0.3007 - val_loss: 16.0377 - val_mse: 0.1736
Epoch 6/80
nomal wmse
238/238 [==============================] - 2s 7ms/step - loss: 19.1687 - mse: 0.2735 - val_loss: 14.4789 - val_mse: 0.1420
Epoch 7/80
nomal wmse
238/238 [==============================] - 2s 7ms/step - loss: 18.5966 - mse: 0.2747 - val_loss: 14.9252 - val_mse: 0.1770
Epoch 8

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
You have provided over 5k background samples! For better performance consider using smaller random sample.
2024-01-29 16:34:54.918864: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 18.57MiB (rounded to 19476480)requested by op BiasAdd
Current allocation summary follows.
2024-01-29 16:34:54.918921: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2024-01-29 16:34:54.918974: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 54, Chunks in use: 53. 13.5KiB allocated for chunks. 13.2KiB in use in bin. 2.5KiB client-requested in use in bin.
2024-01-29 16:34:54.918992: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 7, Chunks in use: 6. 3.5KiB allocated for c

ResourceExhaustedError: OOM when allocating tensor with shape[9510,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:BiasAdd]

<h1>데이터로드</h1>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
print(hist.history.keys())
fig, loss_ax = plt.subplots()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['mse'], 'r', label='train mse')

loss_ax.plot(hist.history['val_loss'], 'b', label='v loss')
loss_ax.plot(hist.history['val_mse'], 'g', label='v mse')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')

loss_ax.legend(loc='upper right')

In [ ]:
x_valid_df

In [ ]:
y_valid_df

In [ ]:
predictions = model.predict(x_valid_df)
predictions

In [ ]:
y_Im_Predict = pd.DataFrame(predictions, columns=['R_Insolation'])
y_Im_Predict

In [ ]:
actuals =y_valid_df
actuals

In [ ]:
time_index = actuals.index
y_Im_Predict['일시'] = time_index
y_Im_Predict

In [ ]:
y_Im_Predict = y_Im_Predict.set_index('일시')
y_Im_Predict

In [ ]:
actuals

In [ ]:


Im_predict = y_Im_Predict['R_Insolation']

fig = plt.figure(figsize=(20,8)) ## 캔버스 생성
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성

ax.plot(actuals['일사(MJ/m2)'], label='true')
ax.plot(Im_predict, label='pred',linestyle = '--')

ax.legend(loc='upper right')
plt.title('Prediction test of jeonju meterorological station irradiance',fontsize = 27)
plt.xlabel('time',fontsize = 27)
plt.ylabel('Irradiance',fontsize = 27)

plt.show()

In [ ]:
loss, mse = model.evaluate(x_valid_df, y_valid_df)
print("Test MSE:", mse)

In [ ]:
rmse = np.sqrt(mse)
ture=actuals.to_numpy()
pred=Im_predict.to_numpy()
error_rate = (rmse / np.mean(ture)) * 100
print("error_rate :", error_rate)

In [ ]:
epsilon = 1e-8
error_rates = np.abs(ture - pred) / (ture + epsilon)
max_error_rate = np.max(error_rates)
min_error_rate = np.min(error_rates)

print(f"Maximum error rate: {max_error_rate}")
print(f"Minimum error rate: {min_error_rate}")

lower_75_percentile_error_rate = np.percentile(error_rates, 20)
upper_75_percentile_error_rate = np.percentile(error_rates, 80)
print(f"Lower 75% error rate: {lower_75_percentile_error_rate}")
print(f"Upper 75% error rate: {upper_75_percentile_error_rate}")

In [ ]:

import shap

In [ ]:
print(filtered_b.values.shape)

<h1> custom loss def</h1>

In [ ]:
shap.initjs()
# SHAP GradientExplainer 초기화
explainer = shap.DeepExplainer(model, filtered_b.values)

# SHAP 값 계산
shap_values = explainer.shap_values(filtered_b.values)
#explainer.expected_value = explainer.expected_value[0]  # 필요한 경우 expected_value를 조정
feature_names = ['temp', 'c_irra', 'hour_cos','hour_sin','month_cos','month_sin','year','humy','cloud','Wx','Wy']
# SHAP 요약 플롯
shap.summary_plot(shap_values, filtered_b.values,feature_names=feature_names)


In [ ]:
explainer = shap.DeepExplainer(model, filtered_b.values)
shap_values = explainer.shap_values(filtered_b.values)
# 계산된 SHAP 값으로 Explanation 객체를 생성합니다.
# 이 부분은 SHAP 라이브러리의 버전에 따라 다를 수 있으니, 해당 문서를 참조하십시오.
feature_names = ['temp', 'c_irra', 'hour_cos','hour_sin','month_cos','month_sin','year','humy','cloud','Wx','Wy']
shap_explanation = shap.Explanation(values=shap_values, data=filtered_b.values,feature_names=feature_names)
print(shap_explanation.shape)

In [ ]:
# 모든 클래스에 대한 평균 SHAP 값 계산
average_shap = np.mean(shap_values, axis=0)
shap_explanation = shap.Explanation(values=average_shap, data=filtered_b.values,feature_names=feature_names)

# 평균 SHAP 값으로 beeswarm 플롯 그리기
shap.plots.beeswarm(shap_explanation)


In [ ]:
shap_values = explainer.shap_values(filtered_b.values)

In [ ]:

shap_values = np.array(shap_values)
shap_values = np.squeeze(shap_values, axis=0)

abs_shap_values = np.abs(shap_values)

In [ ]:
# 절대값에 대한 평균 계산
average_abs_shap_values = np.mean(abs_shap_values, axis=0)

normalized_shap_values = average_abs_shap_values / np.sum(average_abs_shap_values)
print(shap_values.shape)
print(average_abs_shap_values.shape)

print(average_abs_shap_values)
print(normalized_shap_values)

In [ ]:
import numpy as np
import shap

# 예측 모델의 기준값 (일반적으로 모델의 평균 출력)
base_value = model.predict(filtered_b.values).mean()

# 데이터셋의 모든 인스턴스에 대한 SHAP 값의 평균을 계산
average_shap_values = np.mean(shap_values, axis=0)